In [12]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import *
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.models import Model
from keras.utils import np_utils
import os,shutil
import random
import numpy as np
import matplotlib.pyplot as plt
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF, renderPM

In [13]:
data_path = 'Train'

In [14]:
folders = os.listdir(data_path)

In [15]:
print(folders)

['Pikachu', 'Meowth', 'Fearow', 'Dratini', 'Squirtle', 'Aerodactyl', 'Spearow', 'Psyduck', 'Bulbasaur', 'Charmander']


In [11]:
# for f in folders:
#     sub_path = os.path.join(data_path,f)
#     for img in os.listdir(sub_path):
#         img_path = os.path.join(sub_path,img)
#         if img_path[-3:]=='svg':
#             print(img_path)
#             drawing = svg2rlg(img_path)
#             renderPM.drawToFile(drawing, "file.png", fmt="PNG")
#             os.remove(img_path)
#             print(img_path,"has been deleted")

Train/Fearow/8f2d7986ab76482fb76cac6a91b66de4.svg
Train/Fearow/8f2d7986ab76482fb76cac6a91b66de4.svg has been deleted
Train/Fearow/2c8d5221e92444f2b9477350e405bc54.svg
Train/Fearow/2c8d5221e92444f2b9477350e405bc54.svg has been deleted


In [24]:
image_data = []
image_labels = []
image_dict = {
    "Aerodactyl":0,  
    "Bulbasaur":1,  
    "Charmander":2, 
    "Dratini":3,  
    "Fearow":4,  
    "Meowth":5,  
    "Pikachu":6,  
    "Psyduck":7,  
    "Spearow":8,  
    "Squirtle":9 
}

In [25]:
for f in folders:
    sub_path = os.path.join(data_path,f)
    print(sub_path,"is in process")
    for img in os.listdir(sub_path):
        img_path = os.path.join(sub_path,img)
        img = image.load_img(img_path,target_size=((224,224)))
        img_arr = image.img_to_array(img)/255.0
        image_data.append(img_arr)
        image_labels.append(image_dict[f])
    print(sub_path,"HAS BEEN PROCESSED")

Train/Pikachu is in process
Train/Pikachu HAS BEEN PROCESSED
Train/Meowth is in process
Train/Meowth HAS BEEN PROCESSED
Train/Fearow is in process
Train/Fearow HAS BEEN PROCESSED
Train/Dratini is in process
Train/Dratini HAS BEEN PROCESSED
Train/Squirtle is in process
Train/Squirtle HAS BEEN PROCESSED
Train/Aerodactyl is in process
Train/Aerodactyl HAS BEEN PROCESSED
Train/Spearow is in process
Train/Spearow HAS BEEN PROCESSED
Train/Psyduck is in process
Train/Psyduck HAS BEEN PROCESSED
Train/Bulbasaur is in process
Train/Bulbasaur HAS BEEN PROCESSED
Train/Charmander is in process
Train/Charmander HAS BEEN PROCESSED


In [26]:
print(len(image_data),len(image_labels))

1645 1645


In [28]:
total = 0
for f in folders:
    path = os.path.join(data_path,f)
    print(f +" "+ str(len(os.listdir(path))))
    total += len(os.listdir(path))
print(total)

Pikachu 268
Meowth 63
Fearow 109
Dratini 98
Squirtle 248
Aerodactyl 80
Spearow 117
Psyduck 143
Bulbasaur 255
Charmander 264
1645


In [29]:
image_labels[:5]

[6, 6, 6, 6, 6]

In [30]:
combined = list(zip(image_data,image_labels))
random.shuffle(combined)
image_data[:],image_labels[:] = zip(*combined)

In [31]:
print(image_labels[:15])

[6, 6, 1, 1, 1, 0, 6, 5, 6, 3, 9, 5, 9, 4, 9]


In [32]:
X_train = np.array(image_data)
y_train = np_utils.to_categorical(np.array(image_labels))

In [33]:
print(X_train.shape,y_train.shape)

(1645, 224, 224, 3) (1645, 10)


In [ ]:
base_model = MobileNetV2(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [ ]:
base_model.summary()

In [ ]:
gap = GlobalAveragePooling2D()(base_model.output)
fc1 = Dense(256,activation='relu')(gap)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4,activation='softmax')(d1)


In [ ]:
model = Model(inputs = base_model.input,outputs = fc2)

In [ ]:
model.summary()

In [ ]:
adam = Adam(learning_rate=0.00003)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
for ix in range(len(model.layers)):
    print(ix,model.layers[ix])

In [ ]:
for ix in range(69):
    model.layers[ix].trainable = False

In [ ]:
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
hist = model.fit(X_train,y_train,batch_size=16,shuffle=True,epochs=6,validation_split=0.20)

In [ ]:
h = hist.history

In [ ]:
plt.style.use('seaborn')
plt.plot(h['loss'],label='Training Loss')
plt.plot(h['val_loss'],label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.style.use('seaborn')
plt.plot(h['accuracy'],label='Training accuracy')
plt.plot(h['val_accuracy'],label='Validation acc')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()